Limpieza de datos Rooms

In [557]:
import pandas as pd
import numpy as np
import re 

In [784]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [807]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
display(data_room_null_description)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

In [786]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [787]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [788]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [789]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'siete':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [808]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [809]:
resultado = fun_buscar_disminuir(data_room_null_description,r"( [1-9]{1,3}) *(habitaci)",0)
print(resultado)

Disminución en función a los ambientes tipo: ( [1-9]{1,3}) *(habitaci)
True     67687
False    53533
Name: rooms_completo, dtype: int64


In [810]:
resultado = fun_buscar_disminuir(data_room_null_description,r"( [1-9]{1,3}) *(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes tipo: ( [1-9]{1,3}) *(dormitorio|domitorio)
False    74512
True     46708
Name: rooms_completo, dtype: int64


In [811]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+) *(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+) *(dormitorio|domitorio)
False    86080
True     35140
Name: rooms_completo, dtype: int64


In [812]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1,3}) *(ambiente)",1)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1,3}) *(ambiente)
False    96411
True     24809
Name: rooms_completo, dtype: int64


In [813]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+) *(ambiente)",1)
print(resultado)

Disminución en función a los ambientes : (\w+) *(ambiente)
False    98082
True     23138
Name: rooms_completo, dtype: int64


In [814]:
resultado = fun_buscar_disminuir(data_room_null_description,r"( [1-9]{1}) *(amb)",1)
print(resultado)

Disminución en función a los ambientes tipo: ( [1-9]{1}) *(amb)
False    99704
True     21516
Name: rooms_completo, dtype: int64


In [815]:
resultado = fun_buscar_disminuir(data_room_null_description,r"( [1-9]{1}) *(dorm)",0)
print(resultado)


Disminución en función a los ambientes tipo: ( [1-9]{1}) *(dorm)
False    100293
True      20927
Name: rooms_completo, dtype: int64


In [816]:
resultado = fun_buscar_disminuir(data_room_null_description,r"( [1-9]{1}) *(suite)",0)
print(resultado)

Disminución en función a los ambientes tipo: ( [1-9]{1}) *(suite)
False    100403
True      20817
Name: rooms_completo, dtype: int64


In [817]:
data_room_search = data["description"][data["rooms_completo"].isnull()]
data_room_search.sample(8)

88896     Departamento con Agua Corriente, Córdoba, Córd...
84261                                      BBU2956 AP192118
4456      Trenque Lauquen, Casa en alquiler, ubicada a p...
94247     Brooklyn Arias es un complejo residencial de b...
76723     Departamento  PH, en 2 plantas sobre una super...
118539    Departamento 47m² en Defensa 1657, Capital Fed...
80212     Superficie de lote 1300mts – Sector GolfSuperf...
92425     Departamento con Detector de Incendios, Córdob...
Name: description, dtype: object

In [823]:
#76874,100557,35002 = 2
#46551 = 1
#48411 = 4
#66284,75074 = cant, palabra dormitorio
idreg= 52006
print(data.title[idreg])
display(data.loc[idreg:idreg,["property_type","rooms_completo","description"]])
display(data_room_null_description[idreg])
data_finded= fun_buscar_patt(data["description"][data["rooms_completo"].isnull()],"r(dormitorio)")
print("--___----")
display(data_finded[data_finded.notnull()])

EXCELENTE CASA EN 2 PLTAS.(EX-MUNICIPAL)S/LOTE PPIO. 7,33X22


,property_type,rooms_completo,description
52006,house,223.0,EXCELENTE CASA EN 2 PLTAS.(EX-MUNICIPAL) S/LOT...


'excelente casa en 2 pltas.(ex-municipal) s/lote ppio. 7,33x 223 domitorios (1 suite) escritorio o 4 dormitorio-dpcias.-4 baños-jardín al fte.-7,40 x 2,65-jardin/patio al fondo 7,20x6,40.-muy buena ubicacion,a mts. de av. juan bautista alberdi y av. olivera,gran centro comercial,colegios.-cerca de pque. avellaneda.-transportes de colectivos:lineas 4,5,36,49,55,126,180 etc.-'

--___----


Series([], Name: description, dtype: object)

In [819]:
data.loc[idreg:idreg+1,:]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,rooms_completo
50857,50857,sell,house,Paraná,|Argentina|Entre Ríos|Paraná|,Argentina,Entre Ríos,3430146.0,"-31.7192304192,-60.5103358944",-31.719230,...,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19fuu_venta_casa_p...,MUY LINDA PROPIEDAD LISTA PARA ENTRAR A VIVIR!...,MUY LINDA PROPIEDAD LISTA PARA ENTRAR A VIVIR!...,https://thumbs4.properati.com/7/JoqI6XzMHvkYUa...,NaN
50858,50858,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.593156,-58.409715",-34.593156,...,NaN,2506.17284,NaN,3.0,NaN,http://www.properati.com.ar/19fuw_venta_depart...,Corredor Responsable: Hernan Perrone - C.U.C.I...,Oportunidad 3 amb c/ dp a reciclar Apto Credito!,https://thumbs4.properati.com/7/hWUAjfKQiUXJ1h...,3.0


In [820]:
data.property_type.value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

In [821]:
print(data["rooms_completo"].isnull().value_counts())
data_mask = data["rooms_completo"].isnull()
data_encontrada = data_room_null_description.apply(lambda x: x if x is np.NaN else x.count("dormitorio"))
data.loc[data_mask,"rooms_completo"] = data_encontrada[data_mask]
print(data["rooms_completo"].isnull().value_counts())

#data[data_encontrada_mask,"rooms_completo"]= 
#data["rooms_completo"].isnull().value_counts()
# encontradas_mask = encontradas.isnull()
# encontradas[encontradas_mask]
# encontradas

False    100403
True      20817
Name: rooms_completo, dtype: int64
False    121220
Name: rooms_completo, dtype: int64


In [822]:
#display(data.loc[:,["property_type","rooms_completo","description"]])
#data.pivot_table('survived', index='sex', columns='class', margins=True, margins_name= "totales")
#resultado = data.pivot_table(["rooms","rooms_completo"],index='rooms_completo', columns="property_type", margins=True)
vale = (data.loc[:,"rooms_completo"]) > 10
#data.groupby('BARRIO')['M2'].median().sort_values(ascending=False)
data.loc[vale,"rooms_completo"].sort_values(ascending=False)

52006     223.0
79823     145.0
79752     145.0
69799     125.0
69788     125.0
          ...  
114622     11.0
114635     11.0
53000      11.0
115912     11.0
115750     11.0
Name: rooms_completo, Length: 355, dtype: float64